In [1]:
import numpy as np
import itertools
from SALib.test_functions import Ishigami

def ishigami_wrapper(param_values):
    strip_dummy_param = param_values[:, 0:3]
    return Ishigami.evaluate(strip_dummy_param)

In [2]:
problem_with_dummy = {
  'num_vars': 4,
  'names': ['x1', 'x2', 'x3', 'x4'],
  'bounds': [[-np.pi, np.pi]]*4,
  'nominal': [1.0,] * 4
    
}

In [3]:
def oat_sample(problem):
    """Create samples for a 'traditional' OAT result.
    
    Here we assume outputs will be affected at the extreme ends of the nominal bounds.
    
    Parameters
    ==========
    * problem : dict, the problem definition following SALib specs        
    """
    bounds = problem.get('bounds')
    nominal = problem.get('nominal')
    num_vars = problem.get('num_vars')

    experiment = [nominal[:]]
    
    # Samples represent runs with values at the extremes of each parameter boundary
    for i in range(num_vars):
        min_val = bounds[i][0]
        max_val = bounds[i][1]
        
        after = i+1
        
        if i > 0:
            experiment.append(nominal[0:i] + [min_val] + nominal[after:])  # min
            experiment.append(nominal[0:i] + [max_val] + nominal[after:])  # max
        else:
            experiment.append([min_val] + nominal[after:])  # min
            experiment.append([max_val] + nominal[after:])  # max
            
    
    return np.array(experiment)
    
    
def oat_analyze(problem, inputs, results):
    """Analyze a 'traditional' OAT result.
    
    Parameters
    ==========
    * problem : dict, the problem definition following SALib specs
    * inputs : np.array, samples used to generate results. 
               The first row is expected to hold nominal values.
    * results : np.array, model run results obtained using the samples.
                The first row is expected to hold nominal results.          
    """
    X = inputs
    Y = results
    
    num_factors = X.shape[1]
    num_results = Y.size
    nominal_values = problem.get('nominal')

    # Set up result array (ignores one row as it holds the nominal results)
    si_res = np.full(((Y.size - 1),), fill_value=np.nan)
    
    count = 0
    for i in range(num_factors):
        for h in range(1, num_results):
            if X[h][i] == nominal_values[i]:
                # skip nominal input values
                continue

            si_res[count] = (Y[h] - Y[0]) / (X[h][i] - X[0][i])
            count += 1
        # End for
    # End for
    
    return si_res
    

In [4]:
%%time
samples = oat_sample(problem_with_dummy)

Wall time: 0 ns


In [5]:
%time
Y = ishigami_wrapper(samples)

Wall time: 0 ns


In [6]:
%time
oat_analyze(problem_with_dummy, samples, Y)

Wall time: 0 ns


array([ 0.22349327, -0.43221015,  1.19676519, -2.31440555, -1.95879845,
        3.7880898 , -0.        ,  0.        ])